In [1]:
from init import *
import time

In [2]:
model = Model()

In [3]:
m = mcsdk.examples.BuildModel_Simple()
m.evaluations.append(EvaluationPoint(0,2))
m.json()

'{"TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "IndependentBrownianMotion"}, {"name": "BrownianMotion", "start": 0.1, "args": [0.2, 2]}], "evaluations": [{"state": 0, "time": 2}], "RandomSeed": -1}'

In [4]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'mean': [0.4924275279045105],
 'names': ['BrownianMotion'],
 'npaths': [10000],
 'skewness': [-0.016495490446686745],
 'stddev': [2.825214147567749],
 'time_points': [2.0],
 'time_steps': [10]}

In [5]:
m = mcsdk.examples.BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 10000
for t in [1,2,5,10]:
    m.evaluations.append(EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'mean': [0.30380013585090637,
  0.503334105014801,
  1.1016252040863037,
  2.0998497009277344],
 'names': ['BrownianMotion'],
 'npaths': [1000000, 1000000, 1000000, 1000000],
 'skewness': [2.7679961931426078e-05,
  0.0012602308997884393,
  0.0014865552075207233,
  0.00014136405661702156],
 'stddev': [1.9983079433441162,
  2.8288698196411133,
  4.4722371101379395,
  6.321253776550293],
 'time_points': [1.0, 2.0, 5.0, 10.0],
 'time_steps': [1000, 2000, 5000, 10000]}

In [6]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

3.0 nanoseconds to make a single time step


# Two processes, Three evaluations

In [7]:
model = mcsdk.examples.TwoProcessesModel()
model.TimeSteps = 10000
for time in [1,2,5]:
    model.evaluations.append(EvaluationPoint(state=0,time=time))
r = requests.post(f'{server}/model',model.json())
r.json()

{'mean': [0.32174989581108093,
  0.2955664098262787,
  0.5217474699020386,
  0.5015538930892944,
  1.1357734203338623,
  1.0906596183776855],
 'names': ['BrownianMotion', 'BrownianMotion'],
 'npaths': [10000, 10000, 10000, 10000, 10000, 10000],
 'skewness': [0.03348071128129959,
  -0.0008806378464214504,
  0.013001731596887112,
  0.028168419376015663,
  0.0040048351511359215,
  -0.0009979220340028405],
 'stddev': [1.9991837739944458,
  1.9954752922058105,
  2.850731372833252,
  2.8345072269439697,
  4.492213726043701,
  4.48729133605957],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [2000, 4000, 10000]}

In [8]:
er = EvaluationResults(r.json(),model)
er

2 states with 3 evaluations

In [9]:
er.df()

,name,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,0,1.0,2000,10000,0.321750,0.019992,1.999184,0.033481
1,BrownianMotion,1,1.0,2000,10000,0.295566,0.019955,1.995475,-0.000881
2,BrownianMotion,0,2.0,4000,10000,0.521747,0.028507,2.850731,0.013002
3,BrownianMotion,1,2.0,4000,10000,0.501554,0.028345,2.834507,0.028168
4,BrownianMotion,0,5.0,10000,10000,1.135773,0.044922,4.492214,0.004005
5,BrownianMotion,1,5.0,10000,10000,1.090660,0.044873,4.487291,-0.000998
